In [12]:
import requests

url = "http://127.0.0.1:8800"

In [13]:
# get environment coordinates
response = requests.get(f"{url}/scene_objects")
print(response.json())

{'count': 188, 'objects': [{'name': 'bottom_left_group_1_door_handle_main', 'position': {'x': 0.5850066541008743, 'y': -4.099998790818693, 'z': 0.2}, 'quaternion': {'w': 0.7071052811843601, 'x': 0.0, 'y': 0.0, 'z': 0.707108281185553}, 'yaw': 1.5708005694372709}, {'name': 'bottom_left_group_1_door_main', 'position': {'x': 0.5850066541008743, 'y': -4.099998790818693, 'z': 0.2}, 'quaternion': {'w': 0.7071052811843601, 'x': 0.0, 'y': 0.0, 'z': 0.707108281185553}, 'yaw': 1.5708005694372709}, {'name': 'bottom_left_group_1_inner_box', 'position': {'x': 0.30000665410343924, 'y': -4.099999999971769, 'z': 0.2}, 'quaternion': {'w': 0.7071052811843601, 'x': 0.0, 'y': 0.0, 'z': 0.707108281185553}, 'yaw': 1.5708005694372709}, {'name': 'bottom_left_group_1_main', 'position': {'x': 0.3, 'y': -4.1, 'z': 0.2}, 'quaternion': {'w': 0.7071052811843601, 'x': 0.0, 'y': 0.0, 'z': 0.707108281185553}, 'yaw': 1.5708005694372709}, {'name': 'bottom_left_group_2_door_handle_main', 'position': {'x': 0.58500665410087

In [16]:
from pprint import pprint

pprint(response.json())

{'count': 188,
 'objects': [{'name': 'bottom_left_group_1_door_handle_main',
              'position': {'x': 0.5850066541008743,
                           'y': -4.099998790818693,
                           'z': 0.2},
              'quaternion': {'w': 0.7071052811843601,
                             'x': 0.0,
                             'y': 0.0,
                             'z': 0.707108281185553},
              'yaw': 1.5708005694372709},
             {'name': 'bottom_left_group_1_door_main',
              'position': {'x': 0.5850066541008743,
                           'y': -4.099998790818693,
                           'z': 0.2},
              'quaternion': {'w': 0.7071052811843601,
                             'x': 0.0,
                             'y': 0.0,
                             'z': 0.707108281185553},
              'yaw': 1.5708005694372709},
             {'name': 'bottom_left_group_1_inner_box',
              'position': {'x': 0.30000665410343924,
                    

In [18]:
response = requests.get(f"{url}/robot_pose")
pprint(response.json())

{'current': {'theta': 2.2188628741105134e-05,
             'x': -2.007182077539487e-07,
             'y': -7.862209718923823e-08},
 'end_effector': {'position': {'x': 0.6405378900151781,
                               'y': -3.4500116510425256,
                               'z': 1.5565905620819682},
                  'quaternion': {'w': 0.04783608378566806,
                                 'x': 0.8848074470001007,
                                 'y': -0.45402511499228415,
                                 'z': -0.09321312020019432},
                  'yaw': -0.9586773698305512},
 'error': {'theta': -2.2188628741105134e-05,
           'x': 2.007182077539487e-07,
           'y': 7.862209718923823e-08},
 'target': {'theta': 0.0, 'x': 0.0, 'y': 0.0},
 'velocity': {'theta': -2.610038227289871e-07,
              'x': 1.4518888129557344e-10,
              'y': 5.936979025668853e-11}}


In [10]:
payload = {
    "action": {
        "type": "run_code",
        "payload": {
            "code": """set_target_position(0, 0, PI), set_target_position(-0.5, 0, PI)"""
        },
    }
}
response = requests.post(f"{url}/send_action", json=payload)